<a href="https://colab.research.google.com/github/danielbauer1979/MSDIA_PredictiveModelingAndMachineLearning/blob/main/GB888_III_4_RegressionTreesIntro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Regression Trees

In this tutorial, we will introduce regression trees.  We consider a basic simulated example with a single predictor that compares tress to (linear) regression.  In particular, we will demonstrate when trees do well and when the don't.

As usually, let's start with loading the relevant libaries.

In [ ]:
import numpy as np
import pandas as pd
from io import StringIO
import matplotlib.pyplot as plt
import graphviz
import pydot

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor, export_graphviz
from sklearn.metrics import mean_squared_error

# Regression Trees

## Review of Concepts and Maths

### Trees

As we discuss in the previous parts, trees successively split the domains of predictors into two areas (greater and smaller), so that the final tree can be thought of as a model with constant predictions in *square partitions* of the feature domain.  Hence, in building a tree, we need to address two questions:

1. <span style="color:blue"> How to come up with the splits that determine the square partitions? </span>

2. <span style="color:blue"> How to come up with the predictions for each of the square partitions? </span>

The second task is straightforward:  We simply take the average of all predictions in the current partition.  

For 2., one relies on a *greedy algorithm*: For each node, we go over all possible variables and all possible split levels, and then choose the combination of variable and split level that mimizes the prediction error.  However, note that the tree thus will generally not minimize the squared prediction error -- finding such a partition is computationally/algorithmically infeasible...

### Excursion: The Sigmoid Function

In generating our dataset we will use the so-called *sigmoid function*, which will be a key ingredient when we discuss *neural networks*.  It has the advantage that depending on the choice of a parameter, it can mimic a highly linear or a highly non-linear relationship:

$$
\sigma_{\alpha}(x) = \frac{1}{1+e^{-\alpha\,x}}
$$


In [ ]:
def sigmoid(x):
    return(1 / (1 + np.exp(-x)))
x = np.linspace(-5, 5, 1000)
plt.figure(figsize=(15,6))
plt.plot(x, sigmoid(x), color='b')
plt.plot(x, sigmoid(0.5*x), color='r')
plt.plot(x, sigmoid(5*x), color='g')

So we notice that for a small choice of $\alpha$ (red curve), we obtain a relationship that is almost linear (and, indeed, as $\alpha$ becomes smaller and smaller, $\sigma_{\alpha}$ approaches a perfectly linear function).  Whereas for a larger choice of $\alpha$ (green curve), we almost obtain a step function (and, indeed, as $\alpha$ becomes larger and larger, $\sigma_{\alpha}$ approaches a step function).

## A Simulated Example with One Predictor

1. Let's start by generating two datasets based on the sigmoid function with different parameters taken from above.

In [ ]:
np.random.seed(1) #Set a random seed for obtaining the same results every time we run the code
x = 3 * np.random.normal(0, 1, 150)
eps = 0.25 * np.random.normal(0, 1, 150)

y_1 = sigmoid(0.5 * x) + eps
y_2 = sigmoid(5 * x) + eps


Here is the first dataset:

In [ ]:
mydata1 = pd.DataFrame({'y_1':y_1,'x':x})
mytraindata1 = mydata1[1:100]
mytestdata1 = mydata1[101:150]

plt.scatter(mydata1.x,mydata1.y_1)

And here is the second dataset:

In [ ]:
mydata2 = pd.DataFrame({'y_2':y_2,'x':x})
mytraindata2 = mydata2[1:100]
mytestdata2 = mydata2[101:150]
plt.scatter(mydata2.x,mydata2.y_2)

2. And, for comparison, let's fit a linear regression model to both datasets and let's evaluate the out-of-sample error.

In [ ]:
X_1train = mytraindata1['x'].values.reshape(-1,1)
X_1test = mytestdata1['x'].values.reshape(-1,1)
y_1train = mytraindata1['y_1'].values
lmfit1 = LinearRegression(fit_intercept=True)
lmfit1.fit(X_1train,y_1train)
yhat_OOS1 = lmfit1.predict(X_1test)
OLS_OOS_MSE1 = sum((mytestdata1['y_1'] - yhat_OOS1)**2)/50
OLS_OOS_MSE1

In [ ]:
X_2train = mytraindata2['x'].values.reshape(-1,1)
X_2test = mytestdata2['x'].values.reshape(-1,1)
y_2train = mytraindata2['y_2'].values
lmfit2 = LinearRegression(fit_intercept=True)
lmfit2.fit(X_2train,y_2train)
yhat_OOS2 = lmfit2.predict(X_2test)
OLS_OOS_MSE2 = sum((mytestdata2['y_2'] - yhat_OOS2)**2)/50
OLS_OOS_MSE2

3. Let's fit a tree to the second dataset.

In [ ]:
tree2 = DecisionTreeRegressor(max_leaf_nodes=3)
tree2.fit(X_2train, y_2train)

The following function creates images of tree models using pydot, as the package sklearn doesn't offer graphs of the trees


In [ ]:
import pydot
from IPython.display import Image
def print_tree(estimator, features, class_names=None, filled=True):
    tree = estimator
    names = features
    color = filled
    classn = class_names

    dot_data = StringIO()
    export_graphviz(estimator, out_file=dot_data, feature_names=features, class_names=classn, filled=filled)
    graph = pydot.graph_from_dot_data(dot_data.getvalue())
    return(graph)

In [ ]:
graph, = print_tree(tree2, features=['x'])
Image(graph.create_png())

4. Let's use the test data and cross validation to see what size is optimal, and let's prune accordingly.

In [ ]:
from sklearn.model_selection import cross_val_score
OLS_OOS_MSE2 = []
cv_score = []
for i in range(2,6):
    tree2 = DecisionTreeRegressor(max_leaf_nodes=i)
    tree2.fit(X_2train, y_2train)
    ytreehat2 = tree2.predict(X_2test)
    OLS_OOS_MSE2.append(sum((mytestdata2['y_2'] - ytreehat2)**2)/50)
    cv_score.append(sum(cross_val_score(tree2, X_2train, y_2train, cv=5)))

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(range(2, 6), cv_score, marker='o')
axes[0].set_title('Cross-Validation Score')
axes[0].set_xlabel('Max Leaf Nodes')
axes[0].set_ylabel('CV Score')

axes[1].plot(range(2, 6), OLS_OOS_MSE2, marker='o', color='orange')
axes[1].set_title('Out-of-Sample MSE')
axes[1].set_xlabel('Max Leaf Nodes')
axes[1].set_ylabel('MSE')

plt.tight_layout()
plt.show()

So it looks that the simplest tree with only two leaves does best.

In [ ]:
tree2 = DecisionTreeRegressor(max_leaf_nodes=2)
tree2.fit(X_2train, y_2train)
graph, = print_tree(tree2, features=['x'])
Image(graph.create_png())

In [ ]:
ytreehat2 = tree2.predict(X_2test)
test_mse = mean_squared_error(mytestdata2['y_2'], ytreehat2)
print(f"Test MSE for tree2: {test_mse}")

5. Let's redo for the first dataset.

In [ ]:
tree1 = DecisionTreeRegressor()
tree1.fit(X_1train, y_1train)
graph, = print_tree(tree1, features=['x'])
Image(graph.create_png())

In [ ]:
OLS_OOS_MSE1 = []
cv_score = []
for i in range(2,7):
    tree1 = DecisionTreeRegressor(max_leaf_nodes=i)
    tree1.fit(X_1train, y_1train)
    ytreehat1 = tree1.predict(X_1test)
    OLS_OOS_MSE1.append(sum((mytestdata1['y_1'] - ytreehat1)**2)/50)
    cv_score.append(sum(cross_val_score(tree1, X_1train, y_1train, cv=5)))

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(range(2, 7), cv_score, marker='o')
axes[0].set_title('Cross-Validation Score')
axes[0].set_xlabel('Max Leaf Nodes')
axes[0].set_ylabel('CV Score')

axes[1].plot(range(2, 7), OLS_OOS_MSE1, marker='o', color='orange')
axes[1].set_title('Out-of-Sample MSE')
axes[1].set_xlabel('Max Leaf Nodes')
axes[1].set_ylabel('MSE')

plt.tight_layout()  # Adjust layout to prevent overlapping
plt.show()

So ot looks like 4 leaves is optimal. Let's prune:

In [ ]:
tree1 = DecisionTreeRegressor(max_leaf_nodes=4)
tree1.fit(X_1train, y_1train)
graph, = print_tree(tree1, features=['x'])
Image(graph.create_png())

In [ ]:
ytreehat1 = tree1.predict(X_1test)
test_mse = mean_squared_error(mytestdata1['y_1'], ytreehat1)
print(f"Test MSE for tree1: {test_mse}")

Hence, the tree performs very differently between the two datasets:  For the second dataset, the tree provided a very simple (and understandable!) model of the data, and despite the simplicity the prediction accuracy improved relative to the linear regressions.  For the first dataset, on the other hand, the tree is far more complex than the regression model but still provides worse predictions.

Hence, overall, trees are well-suited for modeling non-linear relationships -- which explains their relevance as the foundational structural element for more advanced learners.